In [1]:
## where to write temporary files 
TMP_PREFIX = "tmp/"

## csv file to find the annotations
ANNOTATION_FILE = "toy_data/toy_example_annotation.csv"



mains steps: 
1. check overlap between parts
2. check overlap between cutout and parts extremities: push part extremities 
3. from cut out to sub parts
4. generate split and merge commands
5. apply commands






In [2]:
from collections import defaultdict
import os 
import pandas as pd
import numpy as np

from utils import *

In [3]:
df_annot = pd.read_csv(ANNOTATION_FILE)
df_annot['start_second'] = df_annot.start.apply(time_to_seconds) ## convert time to seconds
df_annot['stop_second'] = df_annot.stop.apply(time_to_seconds)   ## convert time to seconds


## check that all elements start before they stop:
PB = df_annot.loc[ df_annot.stop_second < df_annot.start_second , :]
if PB.shape[0]>0:
    print("PROBLEM: some elements stop before they start! please fix this")
    print( PB )
    
df_annot

,source,start,stop,destination,start_second,stop_second
0,toy_data/part7_clip1.mp4,00:00:00,00:00:30,part1,0,30
1,toy_data/part7_clip1.mp4,00:00:10,00:00:13,OUT,10,13
2,toy_data/part7_clip1.mp4,00:00:32,00:01:40,part2,32,100


In [4]:
## separate parts and cut out
df_annot_parts = df_annot.loc[ df_annot.destination != "OUT" , :].copy().reset_index(drop=True)
df_annot_OUT = df_annot.loc[ df_annot.destination == "OUT" , :].copy().reset_index(drop=True)


In [5]:
## check if some parts overlap :
check_overlap( df_annot_parts )


In [6]:
## merge overlapping cutouts 
df_annot_OUT = merge_overlapping_elements(df_annot_OUT)

##sort cutout by video and start!!
df_annot_OUT.sort_values( by = ['source' , 'start_second']  , inplace=True)


In [7]:
# applying cutouts :
#  * overlaps is across start or stop of part -> move part start or stop
#  * otherwise part is split in 2
df_subparts = associate_parts_and_cutout(df_annot_parts,df_annot_OUT)
df_subparts

,source,start,stop,destination,start_second,stop_second
0,toy_data/part7_clip1.mp4,NaN,NaN,part1,0,10
1,toy_data/part7_clip1.mp4,NaN,NaN,part1,13,30
2,toy_data/part7_clip1.mp4,NaN,NaN,part2,32,100


In [12]:
cmds = create_commands( df_subparts , prefix = TMP_PREFIX )
for part , commands in cmds.items():
    print(f"#part {part}")
    for c in commands:
        print( c )
    

#part part1
ffmpeg -ss 00:00:00 -to 00:00:10 -i toy_data/part7_clip1.mp4 tmp/destination0_clip0.mp4
ffmpeg -ss 00:00:13 -to 00:00:30 -i toy_data/part7_clip1.mp4 tmp/destination0_clip1.mp4
ffmpeg -f concat -safe 0 -i destination0.clipFiles.txt -c copy part1.mp4
rm file 'tmp/destination0_clip0.mp4'
rm file 'tmp/destination0_clip1.mp4'
#part part2
ffmpeg -ss 00:00:32 -to 00:01:40 -i toy_data/part7_clip1.mp4 tmp/destination1_clip0.mp4
ffmpeg -f concat -safe 0 -i destination1.clipFiles.txt -c copy part2.mp4
rm file 'tmp/destination1_clip0.mp4'


In [9]:
# apply commands - this may take a long while 
for part , commands in cmds.items():
    print('creating',part)
    for c in commands:
        os.system(c)

creating part1


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

creating part2


frame= 1700 fps=118 q=-1.0 Lsize=    4595kB time=00:01:08.00 bitrate= 553.5kbits/s speed=4.72x    
video:3962kB audio:580kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.162091%
[libx264 @ 0x555f0571d6c0] frame I:8     Avg QP:13.03  size:169838
[libx264 @ 0x555f0571d6c0] frame P:433   Avg QP:20.97  size:  5258
[libx264 @ 0x555f0571d6c0] frame B:1259  Avg QP:30.41  size:   335
[libx264 @ 0x555f0571d6c0] consecutive B-frames:  0.9%  0.9%  0.5% 97.6%
[libx264 @ 0x555f0571d6c0] mb I  I16..4: 57.7% 19.0% 23.3%
[libx264 @ 0x555f0571d6c0] mb P  I16..4:  0.9%  1.0%  0.4%  P16..4:  1.6%  0.2%  0.2%  0.0%  0.0%    skip:95.7%
[libx264 @ 0x555f0571d6c0] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  1.1%  0.0%  0.0%  direct: 0.2%  skip:98.5%  L0:44.2% L1:53.4% BI: 2.4%
[libx264 @ 0x555f0571d6c0] 8x8 transform intra:32.9% inter:26.1%
[libx264 @ 0x555f0571d6c0] coded y,uvDC,uvAC intra: 15.4% 15.7% 10.0% inter: 0.2% 0.4% 0.1%
[libx264 @ 0x555f0571d6c0] i16 v,h,dc,p: 74% 26%  0%  0%
